In [0]:
dbutils.widgets.text("v_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("v_file_date")

In [0]:
%run "../includes/common_code"

Using race_results from presentation layer to get driver standings

In [0]:
race_results_df = spark.read.format("delta").load(f"{presentation_container_path}/race_results").filter(f"file_date = '{v_file_date}'")

In [0]:
from pyspark.sql.functions import col, when, count, sum, desc, asc

First way to get driver standings but this don't has rank column but shows the correct order

In [0]:
driver_standings_1 = race_results_df \
    .groupBy(race_results_df.race_id,race_results_df.race_year, race_results_df.nationality, race_results_df.driver_name,              race_results_df.team_name) \
    .agg(sum(col('points')).alias('points'), count(when(col('position') == 1, True)).alias('Wins')) \
            .orderBy(desc(col('points')), desc(col('Wins')))

In [0]:
display(driver_standings_1.filter(driver_standings_1.race_year == 2020))

race_id,race_year,nationality,driver_name,team_name,points,Wins


Lets do it using window functions, with this you can get rank

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

In [0]:
driver_standings_2 = race_results_df \
    .groupBy(race_results_df.race_id, race_results_df.race_year, race_results_df.nationality, race_results_df.driver_name, race_results_df.team_name) \
        .agg(sum(col('points')).alias('points'), count(when(col('position') == 1, True)).alias('Wins'))

In [0]:
driver_rank = Window.partitionBy('race_year').orderBy(desc(col('points')), desc(col('Wins')))
driver_standings_2 = driver_standings_2.withColumn('rank', rank().over(driver_rank))

In [0]:
# Lets reorder according to the BBC driver standings page
driver_standings_final = driver_standings_2.select(driver_standings_2.rank, driver_standings_2.nationality, driver_standings_2.driver_name, driver_standings_2.team_name, driver_standings_2.Wins, driver_standings_2.points, driver_standings_2.race_year)

In [0]:
display(driver_standings_final.filter(driver_standings_final.race_year == 2020))

rank,nationality,driver_name,team_name,Wins,points,race_year


In [0]:
merge_condition = "target.driver_name = source.driver_name AND target.race_year = source.race_year"
mergeTable(driver_standings_final, 'f1_presentation', 'driver_standings', merge_condition, 'race_year')

In [0]:
# Lets write this into presentation layer
 # overwrite_partition(driver_standings_final, 'f1_presentation', 'driver_standings', 'race_year')

In [0]:
dbutils.notebook.exit("success")